In [4]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

import os
print(os.listdir(r"C:\Users\Surface Book\OneDrive\Desktop\project"))


['.git', '.gitattributes', 'articulations-robot-demo', 'BBC News Sample Solution.csv', 'BBC News Test.csv', 'BBC News Train.csv', 'p1.ipynb', 'robotiq_2finger_grippers', 'workspace.code-workspace']


In [5]:
TRAIN_PATH = os.path.join(r"C:\Users\Surface Book\OneDrive\Desktop\project", "BBC News Train.csv")

#Load the data using pandas : Create a DataFrame named df, that contains the training data 
df = pd.read_csv(TRAIN_PATH)
df.head()

,ArticleId,Text,Category
0,1833,worldcom ex-boss launches defence lawyers defe...,business
1,154,german business confidence slides german busin...,business
2,1101,bbc poll indicates economic gloom citizens in ...,business
3,1976,lifestyle governs mobile choice faster bett...,tech
4,917,enron bosses in $168m payout eighteen former e...,business


In [6]:
df['category_id'] = df['Category'].factorize()[0]

df['category_id'][0:10]
df.head(20)

,ArticleId,Text,Category,category_id
0,1833,worldcom ex-boss launches defence lawyers defe...,business,0
1,154,german business confidence slides german busin...,business,0
2,1101,bbc poll indicates economic gloom citizens in ...,business,0
3,1976,lifestyle governs mobile choice faster bett...,tech,1
4,917,enron bosses in $168m payout eighteen former e...,business,0
5,1582,howard truanted to play snooker conservative...,politics,2
6,651,wales silent on grand slam talk rhys williams ...,sport,3
7,1797,french honour for director parker british film...,entertainment,4
8,2034,car giant hit by mercedes slump a slump in pro...,business,0
9,1866,fockers fuel festive film chart comedy meet th...,entertainment,4


In [7]:
category_id_df = df[['Category', 'category_id']].drop_duplicates().sort_values('category_id')
category_id_df

,Category,category_id
0,business,0
3,tech,1
5,politics,2
6,sport,3
7,entertainment,4


In [8]:
category_to_id = dict(category_id_df.values)
id_to_category = dict(category_id_df[['category_id', 'Category']].values)
id_to_category


{0: 'business', 1: 'tech', 2: 'politics', 3: 'sport', 4: 'entertainment'}

In [9]:
category_to_id

{'business': 0, 'tech': 1, 'politics': 2, 'sport': 3, 'entertainment': 4}

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')

features = tfidf.fit_transform(df.Text).toarray() 

labels = df.category_id


In [11]:
features.shape 


(1490, 9927)

In [12]:
category_to_id.items()


dict_items([('business', 0), ('tech', 1), ('politics', 2), ('sport', 3), ('entertainment', 4)])

In [13]:
sorted(category_to_id.items())

[('business', 0),
 ('entertainment', 4),
 ('politics', 2),
 ('sport', 3),
 ('tech', 1)]

In [14]:
from sklearn.feature_selection import chi2
N = 3  

for Category, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(features, labels == category_id)                   
  indices = np.argsort(features_chi2[0])                                  
  feature_names = np.array(tfidf.get_feature_names())[indices]           
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]         
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]       
  print("# '{}':".format(Category))
  print("  . Most correlated unigrams:\n       . {}".format('\n       . '.join(unigrams[-N:]))) 
  print("  . Most correlated bigrams:\n       . {}".format('\n       . '.join(bigrams[-N:]))) 


# 'business':
  . Most correlated unigrams:
       . growth
       . bank
       . shares
  . Most correlated bigrams:
       . stock market
       . economic growth
       . analysts said
# 'entertainment':
  . Most correlated unigrams:
       . singer
       . actor
       . film
  . Most correlated bigrams:
       . won best
       . los angeles
       . box office
# 'politics':
  . Most correlated unigrams:
       . blair
       . election
       . labour
  . Most correlated bigrams:
       . prime minister
       . tony blair
       . mr blair
# 'sport':
  . Most correlated unigrams:
       . match
       . coach
       . cup
  . Most correlated bigrams:
       . grand slam
       . champions league
       . australian open
# 'tech':
  . Most correlated unigrams:
       . technology
       . software
       . users
  . Most correlated bigrams:
       . anti virus
       . mobile phones
       . mobile phone


In [15]:
features_chi2

(array([0.13345481, 0.01563765, 0.00108776, ..., 0.00812363, 0.15434356,
        0.120306  ]),
 array([0.71487652, 0.9004835 , 0.97368962, ..., 0.92818297, 0.69441858,
        0.72870284]))

In [16]:
from sklearn.manifold import TSNE

SAMPLE_SIZE = int(len(features) * 0.3)
np.random.seed(0)
indices = np.random.choice(range(len(features)), size=SAMPLE_SIZE, replace=False)          
projected_features = TSNE(n_components=2, random_state=0).fit_transform(features[indices])  
projected_features.shape


(447, 2)

In [17]:
my_id = 0 
projected_features[(labels[indices] == my_id).values]


array([[  2.7825692 ,  15.441475  ],
       [ -7.058563  ,  24.184692  ],
       [  1.2346098 ,  30.813969  ],
       [  9.460215  ,  25.276316  ],
       [ 12.25645   , -19.00246   ],
       [  6.5474377 ,  21.924099  ],
       [-14.644814  ,  17.591993  ],
       [  8.358559  ,  21.490097  ],
       [ 15.427329  ,  25.961151  ],
       [ -2.569004  ,  26.837566  ],
       [-11.073658  ,  23.655098  ],
       [  1.5741967 ,  25.727413  ],
       [ 10.370158  ,  11.785584  ],
       [ -2.226906  ,  22.24362   ],
       [  9.479474  ,  27.850174  ],
       [  2.7283478 ,  24.337986  ],
       [ -6.00327   ,  23.208319  ],
       [  7.298235  ,  23.776398  ],
       [  0.46313453,  19.073269  ],
       [-11.076589  ,  16.818588  ],
       [ 18.201622  ,  20.108046  ],
       [ -4.7741823 ,  16.597239  ],
       [  1.3453362 ,  30.60072   ],
       [ -5.342315  ,  21.719809  ],
       [  5.2986484 ,  23.935171  ],
       [ -7.5837154 ,  19.794918  ],
       [ -3.3005261 ,  18.652798  ],
 

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

from sklearn.model_selection import cross_val_score


models = [
    RandomForestClassifier(n_estimators=200, max_depth=100, random_state=0),
    MultinomialNB(),
    LogisticRegression(random_state=30),
]


In [19]:
CV = 5  

cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = [] 


In [20]:
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))


In [21]:
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])


In [22]:
cv_df.groupby('model_name').accuracy.mean()


model_name
LogisticRegression        0.975168
MultinomialNB             0.972483
RandomForestClassifier    0.955705
Name: accuracy, dtype: float64

In [23]:
from sklearn.model_selection import train_test_split

model = LogisticRegression(random_state=0)

X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(features, labels, df.index, test_size=0.33, random_state=42)

model.fit(X_train, y_train)

y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)


In [31]:
texts = ["AMD announces $8 billion stock buyback program",
"'Love Is Blind' Season 2 Reunion Gets Debut Date",
 "Ballot Rejections in Texas Spike After New Voting Law",
 "Team USA has scored three goals in their hockey game today",
 ""]
text_features = tfidf.transform(texts)
predictions = model.predict(text_features)
for text, predicted in zip(texts, predictions):
  print('"{}"'.format(text))
  print("  - Predicted as: '{}'".format(id_to_category[predicted]))
  print("")


"AMD announces $8 billion stock buyback program"
  - Predicted as: 'business'

"Team USA has scored three goals in their hockey game today"
  - Predicted as: 'sport'

"Ballot Rejections in Texas Spike After New Voting Law"
  - Predicted as: 'politics'

